### SM-2 (SuperMemo)

In [1]:
class SM2:
    def __init__(self):
        self.interval = 0
        self.repetitions = 0
        self.ease_factor = 2.5

    def calculate(self, quality):
        if quality >= 3:
            if self.repetitions == 0:
                self.interval = 1
            elif self.repetitions == 1:
                self.interval = 6
            else:
                self.interval = round(self.interval * self.ease_factor)
            
            self.ease_factor += (0.1 - (5 - quality) * (0.08 + (5 - quality) * 0.02))
            self.ease_factor = max(1.3, self.ease_factor)
            self.repetitions += 1
        else:
            self.interval = 1
            self.repetitions = 0
        
        return self.interval, self.repetitions, self.ease_factor

# Usage example
sm2 = SM2()
interval, repetitions, ease_factor = sm2.calculate(4)
print(f"Next interval: {interval} days")
print(f"Repetitions: {repetitions}")
print(f"Ease factor: {ease_factor:.2f}")


Next interval: 1 days
Repetitions: 1
Ease factor: 2.50


### FSRS

In [2]:
import math

class FSRS:
    def __init__(self):
        self.w = {
            0: 0.40255, 1: 1.18385, 2: 3.173, 3: 15.69105, 4: 7.1949, 5: 0.5345,
            6: 1.4604, 7: 0.0046, 8: 1.54575, 9: 0.1192, 10: 1.01925, 11: 1.9395,
            12: 0.11, 13: 0.29605, 14: 2.2698, 15: 0.2315, 16: 2.9898, 17: 0.51655, 18: 0.6621
        }
        self.decay = -0.5
        self.factor = 19/81
        self.min_difficulty = 1
        self.max_difficulty = 10

    def retrievability(self, t, s):
        return math.pow(1 + self.factor * t / s, self.decay)

    def difficulty(self, d, rating):
        return max(self.min_difficulty, min(self.max_difficulty, d + self.w[0] * (3 - rating)))

    def stability(self, s, d, r, rating):
        hard = 1 if rating == 2 else 0
        easy = 1 if rating == 4 else 0
        new_s = s * (1 + math.exp(self.w[1]) * (11 - d) * math.pow(s, -self.w[2]) * 
                     (math.exp((1 - r) * self.w[3]) - 1) * self.w[4] / (1 + self.w[5] * hard) * 
                     (1 + self.w[6] * easy))
        return max(0.1, new_s)

    def next_interval(self, s):
        return math.ceil(s * 9 / self.factor)

    def calculate(self, difficulty, stability, last_review, now, rating):
        elapsed = (now - last_review).days
        r = self.retrievability(elapsed, stability)
        new_d = self.difficulty(difficulty, rating)
        new_s = self.stability(stability, difficulty, r, rating)
        next_due = now + timedelta(days=self.next_interval(new_s))
        return new_d, new_s, next_due

# Usage example
from datetime import datetime, timedelta

fsrs = FSRS()
difficulty = 5
stability = 2
last_review = datetime(2025, 2, 20)
now = datetime(2025, 2, 25)
rating = 3

new_difficulty, new_stability, next_due = fsrs.calculate(difficulty, stability, last_review, now, rating)
print(f"New difficulty: {new_difficulty:.2f}")
print(f"New stability: {new_stability:.2f}")
print(f"Next review due: {next_due}")


New difficulty: 5.00
New stability: 763.85
Next review due: 2105-05-25 00:00:00
